In [1]:
import numpy as np
from numpy import pi, cos,sin,tan,sqrt
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons

%matplotlib widget

from ipywidgets import *

def fun_n(sigma_t, theta):
    sigma_n = 1/2 * (sigma_t[0,0] + sigma_t[1,1]) + \
              1/2 * (sigma_t[0,0] - sigma_t[1,1]) * cos(2 * theta) + \
              sigma_t[0,1] * sin(2 * theta)
    tau_n   = -1/2 * (sigma_t[0,0] - sigma_t[1,1]) * sin(2 * theta) + \
              sigma_t[0,1] * cos(2 * theta)
    return [sigma_n,tau_n]
    
def fun_sigma_avg(sigma_t):
    sigma_avg = 1/2*(sigma_t[0,0] + sigma_t[1,1])
    return [sigma_avg,0]

def fun_radius(sigma_t):
    radius = sqrt((1/2*(sigma_t[0,0] - sigma_t[1,1]))**2+sigma_t[0,1]**2)
    return radius

def fun_principal_stress(sigma_avg, radius):
    return [[sigma_avg[0]+radius,0],[sigma_avg[0]-radius,0]]


def r(x,d=4):
    return round(x,d)
    
# Initial values (For the problem we will solve)
sigma_x = 10.0
sigma_y = 20.0
tau_xy  = 10.0
theta   = 0.

# Assembly of the stresses into an array
sigma_t          = np.array([[sigma_x,tau_xy],[tau_xy, sigma_y]])
# Calculate the initial quantities
sigma_avg        = fun_sigma_avg(sigma_t)
radius           = fun_radius(sigma_t)
principal_stress = fun_principal_stress(sigma_avg,radius)
[sigma_s, tau_s] = fun_n(sigma_t, theta)

# Start the Mohr's circle interactive plot
fig, ax = plt.subplots()  # Create a figure and an axes.
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.resizable = False

# Plot the mohr's circle
circ = plt.Circle(sigma_avg, radius,edgecolor='#000000', alpha=0.1, facecolor='#a5b8d8')

# Plot the sigma_x, sigma_y, sigma_xy values on the circle 
# (Note these are updated in the update function)
s1,  = ax.plot([sigma_t[1,1], sigma_t[0,0]], [sigma_t[0,1], -sigma_t[0,1]], 'o-', markerfacecolor='#111E6C', c='grey')
sc   = ax.scatter(sigma_avg[0],0, c='#4682B4')
smax = ax.scatter(principal_stress[0][0],0, c='#008081')
smin = ax.scatter(principal_stress[1][0],0, c='#008081')
st,  = ax.plot([sigma_avg[0],sigma_s], [0,-tau_s], 'o-', markerfacecolor='#6593F5', c='grey')

# Here is some boilerplate code on formatting the plot. These are not updated
l = 30 # extent of the grid
ax.set_aspect(1) # set aspect ratio to 1:1
ax.set_xlim([-l,l]) # set x limit
ax.set_ylim([-l,l]) # set x limit

# Move left y-axis and bottim x-axis to centre, passing through (0,0)
ax.spines['left'].set_position('center')
ax.spines['bottom'].set_position('center')

# Eliminate upper and right axes
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')

# Show ticks in the left and lower axes only
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')

# Set x and y label
ax.set_xlabel(r'$\tau_\mathrm{n}$')
ax.set_ylabel(r'$\sigma_\mathrm{n}$')

# Position x and y label
ax.xaxis.set_label_position("top")
ax.yaxis.set_label_position("right")
ax.add_patch(circ)
plt.show()


# Call back function 
def calc(sigma_x=10.0,sigma_y=20.0,tau_xy=10.0,theta=0.):
    sigma_t = np.array([[sigma_x,tau_xy],[tau_xy, sigma_y]])
    sigma_avg        = fun_sigma_avg(sigma_t)
    radius           = fun_radius(sigma_t)
    principal_stress = fun_principal_stress(sigma_avg,radius)
    [sigma_s, tau_s] = fun_n(sigma_t, theta)
    
    print ('sigma: ')
    print (sigma_t)
    print ('sigma average: ')
    print (sigma_avg)
    print ('radius: ')
    print (r(radius))
    print ('principal stress: ')
    print ([r(principal_stress[0][0]),r(principal_stress[1][0])])
    
    print ('max/min shear stress: ')
    print ([r(-radius),r(radius)])

    # Update Mohr's circle
    circ.center = sigma_avg
    circ.radius = radius
    
    s1.set_xdata([sigma_t[1,1], sigma_t[0,0]])
    s1.set_ydata([sigma_t[0,1], -sigma_t[0,1]])
    
    st.set_xdata([sigma_avg[0], sigma_s])
    st.set_ydata([0, -tau_s])
    
    sc.set_offsets([sigma_avg[0],0])
    
    smax.set_offsets([principal_stress[0][0],0])
    smin.set_offsets([principal_stress[1][0],0])
    
    fig.canvas.draw_idle()
    
sigma_x_s = FloatSlider(
                value=10.00,min=-25.00,max=25.00,step=0.5,
                description=r'$\sigma_\mathrm{x}$',
                disabled=False,continuous_update=True)
sigma_y_s = FloatSlider(
                value=20.00,min=-25.00,max=25.00,step=0.5,
                description=r'$\sigma_\mathrm{y}$',
                disabled=False,continuous_update=True)
tau_xy_s = FloatSlider(
                value=10.00,min=-25.00,max=25.00,step=0.5,
                description=r'$\tau_\mathrm{xy}$',
                disabled=False,continuous_update=True)
theta_s = FloatSlider(
                value=0.00,min=0.00,max=1.00,step=1./32.,
                description=r'$\theta$',
                disabled=False,continuous_update=True)

# create the sliders
interact(calc, sigma_x = sigma_x_s, sigma_y = sigma_y_s, 
         tau_xy  = tau_xy_s,theta=theta_s)
